In [11]:
import qforte
from qforte.qkd.qk import qk_energy
from qforte.qkd.mrsqk import mrsqk_energy

# Example notebook for using QForte to perfom Multireference Selected Quantum Krylov calculations

# Must use branch mrsqk_pilot

NOTE: Before running this notebook you will need to have installed Psi4, OpenFermion, and the OpenFermion-Psi4 plugin. There are all avalible via intall through conda-forge and/or pip.

Initialize a QForte molecule, and specify what backend to use. Currently there is only support for using Openfermion-Psi4, but will eventually include direct conections to Psi4 and Forte. First we need to define some input arguments. 

In [4]:
# System factory arguments
system_type = 'molecule'
build_type = 'openfermion'



# Molecule arguments
geometry = [('Be', (0., 0., 0.))]
basis = 'sto-3g'
multiplicity = 1


# Run arguments for backend
run_scf = 1
run_mp2 = 1
run_ccsd = 1
run_fci = 1

We need to initialize an adapter object, wichin in trun is able to ruturn a molecule object.

In [12]:
# Initialize the adapter (specific to backend and system type)
adapter = qforte.system.system_factory(mol_geometry=geometry, basis=basis,
                                       multiplicity=1, description='Be_mrsqk_test')

# Populate the attributes by calling run() with backend run args. we can store the ccsd
# amplitudes 
adapter.run(run_scf=run_scf, run_mp2=run_mp2, run_ccsd=run_ccsd,
                   run_fci=run_fci, store_uccsd_amps=True)


Using standard openfermion hamiltonian ordering!


Get the molecule from the adapter.

In [13]:
Be = adapter.get_molecule()

From the backend, several values are avalible in the molecule for reference.

In [14]:
# Hamiltonian
Hamiltonian = Be.get_hamiltonian()
print('\nQubit Hamiltonian\n', Hamiltonian.str())

# Various energies from psi4 specified in run args
print('\nPsi4 Energies')
print('---------------------------------------------------')
print('hf_energy:                     ', Be.get_hf_energy())
print('mp2_energy:                    ', Be.get_mp2_energy())
print('ccsd_energy:                   ', Be.get_ccsd_energy())
print('fci_energy:                    ', Be.get_fci_energy())


Qubit Hamiltonian
 -9.009402[] +2.243229[Z0] -0.038062[Y0 Z1 Y2] -0.038062[X0 Z1 X2] +2.243229[Z1] -0.038062[Y1 Z2 Y3] -0.038062[X1 Z2 X3] -0.046692[Z2] -0.046692[Z3] -0.148432[Z4] -0.148432[Z5] -0.148432[Z6] -0.148432[Z7] -0.148432[Z8] -0.148432[Z9] +0.568833[Z0 Z1] -0.063333[Y0 Y2] -0.063333[X0 X2] -0.063333[Z0 Y1 Z2 Y3] -0.063333[Z0 X1 Z2 X3] +0.011074[Y0 X1 X2 Y3] -0.011074[X0 X1 Y2 Y3] -0.011074[Y0 Y1 X2 X3] +0.011074[X0 Y1 Y2 X3] +0.003900[Y0 X1 X4 Y5] -0.003900[X0 X1 Y4 Y5] -0.003900[Y0 Y1 X4 X5] +0.003900[X0 Y1 Y4 X5] +0.003900[Y0 X1 X6 Y7] -0.003900[X0 X1 Y6 Y7] -0.003900[Y0 Y1 X6 X7] +0.003900[X0 Y1 Y6 X7] +0.003900[Y0 X1 X8 Y9] -0.003900[X0 X1 Y8 Y9] -0.003900[Y0 Y1 X8 X9] +0.003900[X0 Y1 Y8 X9] +0.131735[Z0 Z2] +0.142809[Z0 Z3] -0.003561[Y0 Z1 Y2 Z3] -0.003561[X0 Z1 X2 Z3] -0.005141[Y0 Z1 Z2 X3 X4 Y5] +0.005141[X0 Z1 Z2 X3 Y4 Y5] +0.005141[Y0 Z1 Z2 Y3 X4 X5] -0.005141[X0 Z1 Z2 Y3 Y4 X5] -0.005141[Y0 Z1 Z2 X3 X6 Y7] +0.005141[X0 Z1 Z2 X3 Y6 Y7] +0.005141[Y0 Z1 Z2 Y3 X6 X7] 

# Multireference Selected Quantum Krylov

This method builds a non-orthoganol Krylov space basis to set up a generalized eigenvalue problem $\bar{\mathbf{H}}\mathbf{C} = \mathbf{S}\mathbf{C}\mathbf{\epsilon}$. The Krylov basis is generated by unitary real-time evolutions of orthogonal reference states. The basis vectors are thus given as $| \Psi_\alpha \rangle = e^{-i n \Delta t \hat{H}}| \Phi_I \rangle$. The elemets of the $\bar{\mathbf{H}}$ and $\mathbf{S}$ matrices can be measured using a quantum computer and the diagonalization performed clasically.   

Here is a sample calculation for a berillium atom which demonstrates energy convergence with increasing number of basis vectors used.

In [24]:
# First we need a reference state (Hartee-Fock usually). 
# In QForte, qubits are aranged as [alpha1, beta1, alpha2, beta2, .... ]
Be_ref = [1,1,1,1,0,0,0,0,0,0]

# parameters for MRSQK
# target root
root = 0

# ∆t
mr_dt = 0.5

# trotter apprximation 
trot_number = 8

# number of evolutions per reference
s = 3

# number of references
d = 3

# parameters of QK (used to select determinants in MR treatment)
Ninit_states = 2
s_0 = 2
dt_0 = 0.25


MRSQK_output = mrsqk_energy(Be, d, s,  mr_dt, Be_ref,
                        trot_order = trot_number,
                        use_phase_based_selection=False,
                        use_spin_adapted_refs=True,
                        target_root=root , 
                        s_0=Ninit_states, 
                        inital_dt=dt_0, 
                        fast=True,
                        print_mats=False, 
                        return_all_eigs=True,
                        return_S=True, 
                        return_Hbar=True)

Emrsqk = MRSQK_output[0]
S = MRSQK_output[2]
Hbar = MRSQK_output[3]

print('\n---------------------------------------------------------')
print('hf_energy:                        ', Be.get_hf_energy())
print('mp2_energy:                       ', Be.get_mp2_energy())
print('ccsd_energy:                      ', Be.get_ccsd_energy())
print('fci_energy:                       ', Be.get_fci_energy())
print('mrsqk_energy:                     ', Emrsqk)





-----------------------------------------------------
        Multreference Selected Quantum Krylov   
-----------------------------------------------------


                   ==> MRSQK options <==
-----------------------------------------------------------
Initial reference:                        |1111000000>
Dimension of reference space (d):         3
Time evolutions per reference (s):        3
Dimension of Krylov space (N):            12
Delta t (in a.u.):                        0.5
Trotter number (m):                       8
Target root:                              0
Use det. selection with sign:             False
Use spin adapted references:              True
Use fast version of algorithm:            True


     ==> Initial QK options (for ref. selection)  <==
-----------------------------------------------------------
Number of initial time evolutions (s_o):  2
Dimension of inital Krylov space (N_o):   3
Initial delta t_o (in a.u.):              0.25


    ==> Initial QK Mat